In [1]:
import pandas as pd
from util.readme_parser import ReadmeSectionParser
from e2e_system.reproder import BinaryHierarchicalReproder, _ground_truth
from pandas import isna
import os
from tqdm.auto import tqdm
import json
from datetime import datetime
import shutil
import glob
from util.file_helper import get_readme_with_repo_name
import numpy as np
import warnings
warnings.filterwarnings("ignore")


In [2]:
neurips = pd.read_csv("data/paperswithcode/neurips_checklist_manuel-100.csv")
neurips.rename(columns={"url": "repo"}, inplace=True)
manuel = pd.read_json(
    "data/acl/sections/labelled/manuel/sections_grouped_orderby-stars/processed/intersection.json")


In [3]:
score_filter = [1, 2, 3, 4]

neurips["score"] = neurips.select_dtypes(include=['bool']).sum(axis=1)
neurips_selected = neurips[neurips.score.isin(score_filter)]

manuel["score"] = manuel.intersection.apply(lambda x: len(x))
manuel_selected = manuel[manuel.score.isin(score_filter)]

selected = pd.concat([neurips_selected, manuel_selected])


In [4]:
selected.sort_values(by="score", ascending=False)

,repo,stars,introduction,requirements,pretrained_model,evaluation,training,results,score,intersection
0,deepmind/lab,6082.0,True,True,False,True,True,False,4,NaN
78,guoyongcs/NAT,41.0,True,True,False,True,True,False,4,NaN
43,microsoft/petridishnn,99.0,True,True,False,True,True,False,4,NaN
44,HazyResearch/hgcn,97.0,True,True,False,False,True,True,4,NaN
45,joansj/blow,95.0,True,True,False,True,True,False,4,NaN
...,...,...,...,...,...,...,...,...,...,...
93,facebookresearch/fastText,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,[requirements]
92,facebookresearch/adaptive-span,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,[requirements]
90,facebookresearch/SpanBERT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,[pretrained_model]
89,facebookresearch/ParlAI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,[requirements]


In [5]:
selected["intersection"].fillna("", inplace=True)


selected["source"] = selected.intersection.apply(
    lambda x: "neurips" if x == "" else "acl")
selected.reset_index(drop=True, inplace=True)


In [9]:
keys = ["header", "parent_header", "content"]
model_dir = "model\\hierarchical_binary2"
reproder = BinaryHierarchicalReproder(model_dir, None)


HFValidationError: Repo id must use alphanumeric chars or '-', '_', '.', '--' and '..' are forbidden, '-' and '.' cannot start or end the name, max length is 96: 'model\hierarchical_binary'.

In [14]:
df = selected.copy()
for repo in (pbar := tqdm(selected.repo.values, desc="Evaluating")):
    pbar.set_postfix_str(f"Evaluating {repo}")
    record = df[df["repo"] == repo]
    readme = get_readme_with_repo_name(df.loc[record.index[0], "source"], repo)
    sections = ReadmeSectionParser(
        repo, readme).parse_sections(group_by_parent=False)

    if len(sections) == 0:
        df.loc[record.index, "reprod_score"] = 0
        continue
    readme_text = ReadmeSectionParser.merge_sections(
        pd.DataFrame(sections), with_newline=True, keys=keys)
    logits = reproder.evaluate(readme_text)
    reprod_score = reproder.calculate_reproducibility(logits, type=2)
    df.loc[record.index, "reprod_score"] = reprod_score


Evaluating:   0%|          | 0/215 [00:00<?, ?it/s]

In [15]:
df

,repo,stars,introduction,requirements,pretrained_model,evaluation,training,results,score,intersection,source,reprod_score
0,deepmind/lab,6082.0,True,True,False,True,True,False,4,,neurips,0.0
1,tensorflow/tpu,3120.0,True,False,False,False,False,False,1,,neurips,0.0
2,tensorflow/lingvo,1914.0,True,True,False,False,True,False,3,,neurips,0.0
3,cornellius-gp/gpytorch,1841.0,True,True,False,False,False,False,2,,neurips,0.0
4,ShangtongZhang/DeepRL,1830.0,True,True,False,True,False,True,4,,neurips,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
210,uma-pi1/kge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,[results],acl,0.0
211,vncorenlp/VnCoreNLP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,[introduction],acl,0.0
212,weizhepei/CasRel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"[introduction, requirements]",acl,0.0
213,wikipedia2vec/wikipedia2vec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,"[introduction, requirements]",acl,0.0
